In [5]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [12]:
# Prepare source dataset 

source_name = 'data/benchmark/Sboost_{sboost}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'count' : 500, 
    'shots' : 1000, 
    'sboost' : 1e12,
    'stepsize' : 0.25, 
    'accepterr': [0.01,0.05,0.1], 
    'iterations_limit' : 100, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'MA-PIA'
}

template = pd.read_csv('data/template_benchmark.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()

log_file, id = log._load_logfile()


Logfile data/benchmark/Sboost_1000000000000.0_ansatz_MA-PIA_stepsize_0.25_shots_1000.csv already exists


In [4]:


id = 100

for i in tqdm.tqdm(range(Config['count']), desc = f"Generating matrices to solve with sboost {Config['sboost']}"):
    
    id += 1
    matrix = np.random.uniform(size = (Config['size'], Config['size']), low = -1, high = 1)
    matrix = (matrix.T + matrix) / 2
    current_datetime = datetime.now()
    data = { 
        'Id (int)' : id, 
        'Valid (int)' : 1, 
        'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
        'Matrix (mat)' : matrix,
        'Size (int)' : Config['size'],
        'Type (str)' : 'Benchmark',
        'Rank (int)' : -1,
        'Density (flt)' : -1.0,
        'Stiffness (flt)' : -1.0, 
        'Penalty (flt)' : -1.0,
        'Sboost (flt)' : Config['sboost'],
        'Test_solver (str)' : '-',
        'Test (flt)' : -1.0,
        'Test_state (vec)' : [],
        'Test_iterations (int)' : -1,
        'Test_time (flt)' : -1.0,
        'Train_solver (str)' : '-',
        'Preprocessing (str)' : '-',
        'Postprocessing (str)' : '-',
        'Train (flt)' : -1.0,
        'Train_state (vec)' : [],
        'Train_iterations (int)' : -1, 
        'Train_iterations_1 (int)' : -1,
        'Train_iterations_2 (int)' : -1,
        'Train_iterations_3 (int)' : -1,
        'Train_shots (int)': -1, 
        'Train_cost (flt)' : -1.0,
        'Train_time (flt)' : -1.0, 
        'Flags (str)' : '',
        'Baren (int)' : -1
    }
    
    log.log(data)


display(log.read(1))

Generating matrices to solve with sboost 100000000.0: 100%|██████████| 500/500 [00:20<00:00, 24.09it/s]


{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-09-01 20:19:38',
 'Matrix (mat)': array([[ 0.33984776,  0.59650208, -0.1313845 ,  0.02527756,  0.54619985,
         -0.17468878,  0.6014892 ,  0.50697242, -0.54844537, -0.13442774],
        [ 0.59650208,  0.10274524, -0.19980245, -0.76346783,  0.37115328,
         -0.14952574,  0.25477122,  0.15227834, -0.35789518, -0.20712598],
        [-0.1313845 , -0.19980245,  0.20562289, -0.62200379,  0.30263077,
          0.37068325, -0.40290254,  0.21045702,  0.30093666, -0.73642429],
        [ 0.02527756, -0.76346783, -0.62200379,  0.31631308,  0.49766738,
          0.023766  , -0.06968882,  0.30389806, -0.06768901,  0.8214402 ],
        [ 0.54619985,  0.37115328,  0.30263077,  0.49766738, -0.93073043,
         -0.56788041, -0.09672308,  0.4594314 ,  0.28633457, -0.80961989],
        [-0.17468878, -0.14952574,  0.37068325,  0.023766  , -0.56788041,
         -0.70894234, -0.05804103,  0.62861839, -0.11262834, -0.20494556],
        [ 0.60

In [6]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])

In [7]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)

        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

Columns in data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other 

KeyboardInterrupt: 

In [8]:


ids = range (101,601)
cores = 25
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113,
        114, 115, 116, 117, 118, 119, 120],
       [121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140],
       [141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160],
       [161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
        174, 175, 176, 177, 178, 179, 180],
       [181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193,
        194, 195, 196, 197, 198, 199, 200],
       [201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 219, 220],
       [221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233,
        234, 235, 236, 237, 238, 239, 240],
       [241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
        254, 255, 256, 257, 258, 259, 260],
       [261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 2

{'Id (int)': 421, 'Valid (int)': 1, 'Updated (str)': '2025-09-02 20:06:06', 'Matrix (mat)': array([[ 0.39761977,  0.34419992,  0.21026534,  0.54088876,  0.27034783,
         0.69727777,  0.09385891, -0.84280318, -0.31355441, -0.13768184],
       [ 0.34419992,  0.82936937,  0.19504586,  0.43085985,  0.46799011,
         0.34894691, -0.29512038,  0.18802961,  0.31530217, -0.46375401],
       [ 0.21026534,  0.19504586, -0.24784482,  0.6181955 ,  0.10510821,
        -0.22692398, -0.09388546, -0.55929896,  0.61536539,  0.33171838],
       [ 0.54088876,  0.43085985,  0.6181955 ,  0.6651858 ,  0.27114895,
        -0.65219131, -0.29792978,  0.29809331,  0.24483962,  0.55672806],
       [ 0.27034783,  0.46799011,  0.10510821,  0.27114895, -0.26801478,
         0.57085842,  0.41208091, -0.41646329, -0.41822448,  0.40044987],
       [ 0.69727777,  0.34894691, -0.22692398, -0.65219131,  0.57085842,
        -0.96500519, -0.07577375,  0.75104242, -0.76352992, -0.42030049],
       [ 0.09385891, -0.29

In [13]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = True, sboosted = True, sboost = line['Sboost (flt)'])
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        trained_data['Sboost (flt)'] = Config['sboost']
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [14]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 40

solvers = [] 

depth = 3

template = pd.read_csv('data/template_benchmark.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)

ids = np.array(range(1, 601))
ids = ids.reshape(cores, len(ids) // cores) 

data = [[log.read(id) for id in subset] for subset in ids]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




1000181651653.5631
1000039608035.78821000035151979.4116
1146229219538.2856
1000129105534.2917

1000300617473.64951000126919861.51371000005976914.2556
1107642769543.7708

999999591801.7816999899739412.87721036454396222.88431163425736083.239
1000000232010.2716999683177894.7557
1000062189770.4474

1000044355583.2268
1000005224808.409999941247209.1426999971270532.1893



1000005760623.4525
1000061570007.7891064081352440.969

999765350528.55921000044874941.866
1274581210328.1328999945088730.1495

999954195253.798999762711483.6498

1006760480752.4158999829648184.26731000120670990.6647


1002591066730.1461

1000133392816.9349999927966404.1998999973027330.3508


999980352341.1167



999964663308.4121000068022919.0745

999637796482.6353
Ansatz: MA-PIA Iteration: 1 | State: [ 0.346  0.522 -0.106  0.464 -0.106 -0.288 -0.618  0.092  0.096  0.27 ] | Sol: -4.551786778674594 | Cursol: -2.623476376311959 | Curenergy: -1.2072037142679093 | Preenergy: 0 | Baren_queue: [] | 100000000.0
Ansatz: MA-PIA Ite

Process ForkPoolWorker-117:
Process ForkPoolWorker-109:
Exception ignored in: Process ForkPoolWorker-108:
Process ForkPoolWorker-99:
Process ForkPoolWorker-114:
Process ForkPoolWorker-89:
Process ForkPoolWorker-90:
Process ForkPoolWorker-107:
Process ForkPoolWorker-102:
Process ForkPoolWorker-118:
Process ForkPoolWorker-113:
Process ForkPoolWorker-93:
Process ForkPoolWorker-101:
Process ForkPoolWorker-100:
Process ForkPoolWorker-110:
Process ForkPoolWorker-105:
Process ForkPoolWorker-119:
Process ForkPoolWorker-103:
Process ForkPoolWorker-116:


KeyboardInterrupt: 

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
